### 库导入

In [3]:
import cv2
import numpy as np  # 修正：numpy np
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

# from scripts.regsetup import description

In [4]:
from lib.imgproc import to_jpeg_bytes, process_frame

### 自定义函数

In [5]:
# 关闭按钮逻辑
def close_video(b):
    global playing, locked_frame
    playing = False
    locked_frame = None
    toggle_button.value = False
    toggle_button.disabled = True
    close_button.disabled = True
    slider_red1_h.disabled = True
    slider_red2_h.disabled = True
    slider_red1_s.disabled = True
    slider_red2_s.disabled = True
    slider_red1_v.disabled = True
    slider_red2_v.disabled = True
    cap.release()
    # 清除画面
    img_processed.value = b''
    img_mask_origin.value = b''
    img_mask_eroded.value = b''
    img_mask_dilated.value = b''


In [6]:
# 播放/暂停按钮
# 引入全局变量 -> 获取change最新的值并赋值 -> 更新图标 + 继续播放/暂停
def toggle_play(change):
    global playing
    playing = change['new']# 获取change最新的值
    toggle_button.icon = 'pause' if playing else 'play'
    if playing:
        threading.Thread(target=play_video, daemon=True).start()

In [7]:
# playing为1，连续循环播放图片
def play_video():
    global playing, locked_frame
    while playing:
        with frame_lock:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)# 播放完毕，从头开始重新播放
                continue
            temp = [
                [slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_v.value[0]],  # 修正：red1_v
                [slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_v.value[1]],  # 修正：red1_v
                [slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]],
                [slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_v.value[1]],
            ]
            draw ,mask ,mask8,mask9= process_frame(frame,temp,lowest=slider_lowest.value)
            img_processed.value = to_jpeg_bytes(draw, is_mask=False)
            img_mask_origin.value = to_jpeg_bytes(mask, is_mask=True)
            img_mask_eroded.value = to_jpeg_bytes(mask8, is_mask=True)
            img_mask_dilated.value = to_jpeg_bytes(mask9, is_mask=True)
            locked_frame = frame.copy()  # 每次播放时更新锁定帧
        #----------------控制播放速度------------------------
        time.sleep(1 / max(1, frame_rate.value))  # 修正：防止除零

In [8]:
# 滑块变化时更新当前帧
def refresh_on_change(change=None):
    global locked_frame
    with frame_lock:
        if playing:
            # 回退一帧，因为你用play_video函数读取了一帧，cv2.CAP_PROP_POS_FRAMES++,所以这里需要回退一下
            current = cap.get(cv2.CAP_PROP_POS_FRAMES)
            cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, current - 1))
            ret, frame = cap.read()
            if ret:
                locked_frame = frame.copy()
                temp = [
                    [slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_v.value[0]],  # 修正：red1_v
                    [slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_v.value[1]],  # 修正：red1_v
                    [slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]],
                    [slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_v.value[1]],
                ]
                draw ,mask ,mask8, mask9= process_frame(frame,temp,lowest=slider_lowest.value)
                img_processed.value = to_jpeg_bytes(draw, is_mask=False)
                img_mask_origin.value = to_jpeg_bytes(mask, is_mask=True)
                img_mask_eroded.value = to_jpeg_bytes(mask8, is_mask=True)
                img_mask_dilated.value = to_jpeg_bytes(mask9, is_mask=True)
        else:
            # 暂停时只对locked_frame做处理，不再读取新帧
            frame = locked_frame
            if frame is not None:
                temp = [
                    [slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_v.value[0]],  # 修正：red1_v
                    [slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_v.value[1]],  # 修正：red1_v
                    [slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]],
                    [slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_v.value[1]],
                ]
                draw ,mask ,mask8, mask9= process_frame(frame,temp,lowest=slider_lowest.value)
                img_processed.value = to_jpeg_bytes(draw, is_mask=False)
                img_mask_origin.value = to_jpeg_bytes(mask, is_mask=True)
                img_mask_eroded.value = to_jpeg_bytes(mask8, is_mask=True)
                img_mask_dilated.value = to_jpeg_bytes(mask9, is_mask=True)

### 全局变量定义

In [9]:
playing = False
locked_frame = None
frame_lock = threading.Lock()

### 读取视频

In [10]:
cap = cv2.VideoCapture(0)#"video/red-green.mp4")
if not cap.isOpened():
    raise IOError("无法打开视频文件")

### 控件定义

In [11]:
# -------------------------------图像输出--------------------------
# 掩膜图像 + 处理后的图像
# 格式为 jpg , 大小为640*480
img_processed = widgets.Image(format='jpg', width=640, height=480)
img_mask_origin = widgets.Image(format='jpg', width=640, height=480)
img_mask_eroded = widgets.Image(format='jpg', width=640, height=480)
img_mask_dilated = widgets.Image(format='jpg', width=640, height=480)
# -------------------------------控制按钮-------------------------
# 切换按钮 + 停止按钮
toggle_button = widgets.ToggleButton(description="Play/Pause", value=False, icon='play')
close_button = widgets.Button(description="关闭", icon='times', button_style='danger')
toggle_button.observe(toggle_play, names='value')
close_button.on_click(close_video)
# -------------------------------滑块控件------------------------
slider_lowest = widgets.IntSlider(value=30, min=0, max=100, description='lowest:')
frame_rate = widgets.IntSlider(value=30, min=1, max=200, description='frame rate:')  # 修正：min=1防止除零
slider_red1_h = widgets.IntRangeSlider(value=[0, 15], min=0, max=180, description='red1 H:')
slider_red1_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red1 S:')
slider_red1_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red1 V:')
slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, description='red2 H:')
slider_red2_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red2 S:')
slider_red2_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red2 V:')


### 布局

In [12]:
group0 = widgets.HBox([toggle_button, close_button])
group1 = widgets.HBox([slider_red1_h, slider_red2_h])
group2 = widgets.HBox([slider_red1_s, slider_red2_s])
group3 = widgets.HBox([slider_red1_v, slider_red2_v])
controls = widgets.VBox([group0, frame_rate,slider_lowest,group1, group2, group3])
image0 = widgets.HBox([img_processed, img_mask_origin])
image1 = widgets.HBox([img_mask_eroded, img_mask_dilated])

### 绑定回调函数

In [13]:
for s in [slider_lowest, slider_red1_h, slider_red2_h, slider_red1_s ,slider_red2_s , slider_red1_v,slider_red2_v]:
    # 当s.value变化时，调用refresh_on_change函数
    s.observe(refresh_on_change, names='value')

In [14]:
display(controls, image0, image1)
refresh_on_change()

In [30]:
cap.release()
